### Objective:

Define classes that have fields that need validation before setting their values. Two data descriptors are required:

* IntegerField: only allow integral numbers, between minimum and maximum value
* CharField: only allow strings within minimum and maximum length

Create a `BaseValidator` for validation and let above two descriptors inherit from it.

---

### Implementation

In [1]:
import numbers

In [2]:
class BaseValidator:
    def __init__(self, min_=None, max_=None):
        self._min = min_
        self._max = max_
        
    def __set_name__(self, owner_class, prop_name):
        self.prop_name = prop_name
    
    def validate(self, value):
        # Should be overwritten by subclass
        pass
          
    def __set__(self, instance, value):
        self.validate(value)
        instance.__dict__[self.prop_name] = value
                             
    def __get__(self, instance, owne_class):
        if instance is None:
            return self
        return instance.__dict__.get(self.prop_name, None)    

In [3]:
class IntegerField(BaseValidator):
    def validate(self, value):
        if not isinstance(value, numbers.Integral):
            raise ValueError(f'{self.prop_name} must be an integer') 
        if self._min is not None and value < self._min:
            raise ValueError(f'{self.prop_name} must be >= {self._min}')
        if self._max is not None and value > self._max:
            raise ValueError(f'{self.prop_name} must be <= {self._max}')

In [4]:
class CharField(BaseValidator):
    def __init__(self, min_, max_):
        min_ = max(min_ or 0, 0)
        super().__init__(min_, max_)
        
    def validate(self, value):
        if not isinstance(value, str):
            raise ValueError(f'{self.prop_name} must be a string') 
        if self._min is not None and len(value) < self._min:
            raise ValueError(f'{self.prop_name} must be >= {self._min} chars')
        if self._max is not None and len(value) > self._max:
            raise ValueError(f'{self.prop_name} must be <= {self._max} chars')
        

In [5]:
class Person:
    name = CharField(min_=0, max_=None)
    age = IntegerField(min_=0, max_=130)

In [6]:
p = Person()
try:
    p.name = 1
except Exception as ex:
    print(ex)

try:
    p.age = -1
except Exception as ex:
    print(ex)

name must be a string
age must be >= 0
